# CNN-LSTM for Predicting Household Power Consumption
- In this problem, we will use a combined architecture of CNN and LSTM to predict household power consumption from historical power consumption.
- The data is provided in the "household_power_consumption.txt" file.
- In this dataset, one household's electric power consumption measurements have a one-minute sampling rate over almost 4 years.
- Unlike single-time step prediction, we are now interested in predicting 60-time points (1 hour) from 600-time points (10 hours).
- Note that we must carefully tune the learning rate and number of epochs.

In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import itertools
import os
from tqdm import tqdm
import json

In [2]:
data_directory = r"C:\Users\said_\OneDrive\Masaüstü\github\Machine Learning in Finance\Datasets"

In [3]:
file_name = os.path.join(data_directory, "household_power_consumption.txt")
power_df = pd.read_csv(file_name, sep=';', low_memory=False, na_values=['?'])
print(power_df.shape)

(2075259, 9)


In [4]:
power_df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [5]:
# Convert the 'Date' column to datetime format
power_df['Date'] = pd.to_datetime(power_df['Date'], format='%d/%m/%Y')

# Create day, month, and year columns
power_df['Day'] = power_df['Date'].dt.day
power_df['Month'] = power_df['Date'].dt.month
power_df['Year'] = power_df['Date'].dt.year

# Convert the 'Time' column to datetime
power_df['Time'] = pd.to_datetime(power_df['Time'], format='%H:%M:%S')

# Extract hours and minutes
power_df['Hour'] = power_df['Time'].dt.hour
power_df['Minute'] = power_df['Time'].dt.minute

# Check if the time extraction is proper
for c in ["Day", "Month", "Year", "Hour", "Minute"]:
    date_indices = power_df[c].value_counts().index.to_list()
    print(f"Length of {c}: {len(date_indices)}")

Length of Day: 31
Length of Month: 12
Length of Year: 5
Length of Hour: 24
Length of Minute: 60


In [6]:
# Check for missing values
print(power_df.isnull().sum())

Date                         0
Time                         0
Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
Day                          0
Month                        0
Year                         0
Hour                         0
Minute                       0
dtype: int64


In [7]:
# Replace missing values using forward fill
power_df = power_df.ffill()
# If there are still NaN values (e.g., at the start of the series), optionally use backward fill
power_df = power_df.bfill()
# Check for missing values once more
print(power_df.isnull().sum())

Date                     0
Time                     0
Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
Day                      0
Month                    0
Year                     0
Hour                     0
Minute                   0
dtype: int64


In [8]:
# Re-order the data columns as target + features
target = ['Global_active_power']
main_features = [
    'Global_reactive_power', 'Voltage', 'Global_intensity', 
    'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 
]
# Do not include some date time features
time_features = ['Month', 'Day', 'Hour']
# Make sure the target is the first column
column_order = target + main_features + time_features
final_df = power_df[column_order].copy()
print(final_df.shape)

(2075259, 10)


In [9]:
final_df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Month,Day,Hour
0,4.216,0.418,234.84,18.4,0.0,1.0,17.0,12,16,17
1,5.360,0.436,233.63,23.0,0.0,1.0,16.0,12,16,17
2,5.374,0.498,233.29,23.0,0.0,2.0,17.0,12,16,17
3,5.388,0.502,233.74,23.0,0.0,1.0,17.0,12,16,17
4,3.666,0.528,235.68,15.8,0.0,1.0,17.0,12,16,17


In [10]:
# Normalize data
new_df = final_df.copy()
# Specify the column to be excluded from scaling
column_not_scaled = 'Global_active_power'
# Select only the numeric columns (exclude any non-numeric ones)
numeric_columns = new_df.select_dtypes(include=[np.number]).columns.tolist()
# Exclude the target column ('Earnings Per Share') from scaling
columns_to_scale = [col for col in numeric_columns if col != column_not_scaled]
# Extract the subset of the numeric columns to scale
subset_to_scale = new_df[columns_to_scale]
# Initialize and fit the scaler on the subset
scaler = StandardScaler()
scaled_subset = scaler.fit_transform(subset_to_scale)
# Replace the original values with scaled values in the DataFrame
new_df[columns_to_scale] = scaled_subset
print(new_df.shape)

(2075259, 10)


In [11]:
new_df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Month,Day,Hour
0,4.216,2.618973,-1.854882,3.116440,-0.181657,-0.049761,1.257014,1.624716,0.025759,0.794432
1,5.360,2.778952,-2.228850,4.155571,-0.181657,-0.049761,1.138242,1.624716,0.025759,0.794432
2,5.374,3.329993,-2.333932,4.155571,-0.181657,0.123044,1.257014,1.624716,0.025759,0.794432
3,5.388,3.365544,-2.194853,4.155571,-0.181657,-0.049761,1.257014,1.624716,0.025759,0.794432
4,3.666,3.596626,-1.595268,2.529104,-0.181657,-0.049761,1.257014,1.624716,0.025759,0.794432


In [ ]:
# Create sequences
sequences = list()
targets = list()

# Define a sequence and output lengths in minutes
# Sequence length: 600 mins (10 hours)
sequence_length = 600
# Predicting the next 60 time points (1 hour)
output_length = 60

for i in range(len(new_df) - sequence_length - output_length + 1):
    # Create sequence of 600 time points as input features
    seq_features = new_df.iloc[i:(i + sequence_length)].values
    # Create target sequence of 60 time points (next 60 time steps)
    seq_target = new_df.iloc[(i + sequence_length):(i + sequence_length + output_length), 0].values
    
    sequences.append(seq_features)
    targets.append(seq_target)

# Convert sequences and targets to numpy arrays
X = np.array(sequences, dtype=np.float32)
y = np.array(targets, dtype=np.float32)

print(f"Sequence shape: {X.shape}")
print(f"Target shape: {y.shape}")

In [ ]:
# Save sequences and target for future use
np.save(os.path.join(data_directory, 'power_sequences.npy'), X)
np.save(os.path.join(data_directory, 'power_targets.npy'), y)

## Model Development: CNN+LSTM

In [12]:
# Load sequences and targets
X = np.load(os.path.join(data_directory, 'power_sequences.npy'))
y = np.load(os.path.join(data_directory, 'power_targets.npy'))

print(f"Sequence shape: {X.shape}")
print(f"Target shape: {y.shape}")

Sequence shape: (2074600, 600, 10)
Target shape: (2074600, 60)


In [13]:
# Define a hybrid architecture
model = Sequential()

# CNN layers for feature extraction
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(600, 10)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# LSTM layers for capturing temporal dependencies
model.add(LSTM(units=64, return_sequences=False))
model.add(Dropout(0.2))

# Fully connected layers for prediction
model.add(Dense(120, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(60))  # Output layer for the next 60 time steps

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 598, 64)           1984      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 299, 64)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 297, 128)          24704     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 148, 128)         0         
 1D)                                                             
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dropout (Dropout)           (None, 64)                0

#### Hyperparameter Tuning

In [ ]:
# Train-validation-test split
train_size = 0.8
val_size = 0.1
test_size = 0.1

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=(val_size + test_size), random_state=42,
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=(test_size / (val_size + test_size)), random_state=42,
)

print(f"Train shape: {X_train.shape}")
print(f"Validation shape: {X_val.shape}")
print(f"Test shape: {X_test.shape}")

In [ ]:
# Create scenarios for hyperparameter tuning
lr_list = [1e-3, 5e-3, 1e-2]
epoch_list = [5, 10, 15, 20]

# Generate all combinations of learning rate and epochs
power_combinations = list(itertools.product(lr_list, epoch_list))
print(f"Number of combinations: {len(power_combinations)}")
print(power_combinations)

In [ ]:
# Check if the results file already exists
results_file = os.path.join(data_directory, "hp_results_power.csv")

# Initialize an empty dataframe if the file doesn't exist
if os.path.exists(results_file):
    # Load existing results if the file exists
    results_df = pd.read_csv(results_file)
else:
    # Create an empty df to store results
    results_df = pd.DataFrame(columns=["Scenario", "Learning Rate", "Number of Epochs", "MAPE"])

scenario_id = 1

for combo in tqdm(power_combinations):

    # Define parameter values from combinations
    lr, epoch = combo

    # Compile the model with the given hyperparameters
    model.compile(optimizer=Adam(learning_rate=lr), loss="mae", metrics=["mae"])

    # Define EarlyStopping to prevent overfitting
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
    ) 
    
    # Train the model
    model.fit(
        X_train, 
        y_train,
        validation_data=(X_val, y_val), # Calculate val_loss for early stopping
        epochs=epoch, 
        batch_size=2560, 
        verbose=0, 
        callbacks=[early_stopping],
    )

    # Make predictions over the validation set and evaluate model's performance using MAPE
    preds = model.predict(X_val, verbose=0)
    mape = mean_absolute_percentage_error(y_val, preds)

    # Store scenario results in a dictionary
    result = {
        "Scenario": f"S{scenario_id}",
        "Learning Rate": lr,
        "Number of Epochs": epoch,
        "MAPE": mape,
    }

    scenario_id += 1

    # Convert the result to a DataFrame
    scenario_result = pd.DataFrame([result])

    # Concatenate the new result with the existing DataFrame
    if results_df.empty:
        results_df = scenario_result
    else:
        results_df = pd.concat([results_df, scenario_result], ignore_index=True)

    # Save results to CSV after each scenario
    results_df.to_csv(results_file, index=False)

#### Model Evaluation

In [14]:
# Use 90% of data for training and 10% for testing
test_size = 0.1

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=42,
)

print(f"Shape of training sequences: {X_train.shape}")
print(f"Shape of test sequences: {X_test.shape}")

Shape of training sequences: (1867140, 600, 10)
Shape of test sequences: (207460, 600, 10)


In [15]:
# Compile the model with the given hyperparameters
model.compile(
    optimizer=Adam(learning_rate=5e-3), 
    loss="mae", 
    metrics=["mae"],
) 

# Train the model
model.fit(
    X_train, 
    y_train,
    epochs=10, 
    batch_size=2560, 
    verbose=1, 
)

Epoch 1/10
730/730 [==============================] - 3806s 5s/step - loss: 0.4171 - mae: 0.4171
Epoch 2/10
730/730 [==============================] - 3375s 5s/step - loss: 0.3853 - mae: 0.3853
Epoch 3/10
730/730 [==============================] - 3355s 5s/step - loss: 0.3732 - mae: 0.3732
Epoch 4/10
730/730 [==============================] - 2964s 4s/step - loss: 0.3632 - mae: 0.3632
Epoch 5/10
730/730 [==============================] - 3374s 5s/step - loss: 0.3543 - mae: 0.3543
Epoch 6/10
730/730 [==============================] - 3554s 5s/step - loss: 0.3463 - mae: 0.3463
Epoch 7/10
730/730 [==============================] - 3594s 5s/step - loss: 0.3405 - mae: 0.3405
Epoch 8/10
730/730 [==============================] - 3572s 5s/step - loss: 0.3364 - mae: 0.3364
Epoch 9/10
730/730 [==============================] - 3590s 5s/step - loss: 0.3327 - mae: 0.3327
Epoch 10/10
730/730 [==============================] - 3571s 5s/step - loss: 0.3309 - mae: 0.3309


In [16]:
# Make predictions over the validation set and evaluate model's performance using MAPE
test_preds = model.predict(X_test, verbose=1)
mape = mean_absolute_percentage_error(y_test, test_preds)
mse = mean_squared_error(y_test, test_preds)
r2 = r2_score(y_test, test_preds)

# Store test results in a dictionary
test_result = {
    "Test MAPE": mape,
    "Test MSE": mse,
    "Test R2": r2,
}

print(test_result)

6484/6484 [==============================] - 229s 35ms/step
{'Test MAPE': 0.3501448631286621, 'Test MSE': 0.37731996178627014, 'Test R2': 0.6575421094894409}


In [19]:
# Save test results as a JSON file
# Define a custom function to handle numpy types
def handle_numpy_types(obj):
    if isinstance(obj, np.float32):
        return float(obj)  # Convert numpy.float32 to Python float
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

# Save the dictionary as a JSON file with the custom handler
output_file = "test_result.json"
with open(output_file, "w") as f:
    json.dump(test_result, f, indent=4, default=handle_numpy_types)

**Discussion:** A large amount of training data hindered extensive hyperparameter tuning, which resulted in low regression accuracy with a MAPE of approximately 35%. Adjusting the architecture of the hybrid CNN-LSTM model and experimenting with different combinations of learning rates and epoch numbers could improve the regression performance.

# END